In [1]:
vars()

{'In': ['', u'vars()'],
 'Out': {},
 'SQLContext': pyspark.sql.context.SQLContext,
 'SparkContext': pyspark.context.SparkContext,
 'SparkSession': pyspark.sql.session.SparkSession,
 'StorageLevel': pyspark.storagelevel.StorageLevel,
 '_': '',
 '__': '',
 '___': '',
 '__builtin__': <module '__builtin__' (built-in)>,
 '__builtins__': <module '__builtin__' (built-in)>,
 '__doc__': '\nAn interactive shell.\n\nThis file is designed to be launched as a PYTHONSTARTUP script.\n',
 '__name__': '__main__',
 '__package__': None,
 '_dh': [u'/home/gpadmin/spark'],
 '_i': u'',
 '_i1': u'vars()',
 '_ih': ['', u'vars()'],
 '_ii': u'',
 '_iii': u'',
 '_oh': {},
 '_pythonstartup': '',
 '_sh': <module 'IPython.core.shadowns' from '/opt/anaconda/lib/python2.7/site-packages/IPython/core/shadowns.pyc'>,
 'atexit': <module 'atexit' from '/opt/anaconda/lib/python2.7/atexit.pyc'>,
 'exit': <IPython.core.autocall.ZMQExitAutocall at 0x7f72ddfa7310>,
 'get_ipython': <bound method ZMQInteractiveShell.get_ipython o

In [2]:
# echo "export SPARK_CLASSPATH=/usr/share/java/postgresql-jdbc.jar" >> ${SPARK_HOME}/bin/load-spark-env.sh

sc.getConf().getAll()

[(u'spark.sql.catalogImplementation', u'hive'),
 (u'spark.rdd.compress', u'True'),
 (u'spark.driver.host', u'10.0.0.10'),
 (u'spark.executor.extraClassPath', u'/usr/share/java/postgresql-jdbc.jar'),
 (u'spark.driver.port', u'35222'),
 (u'spark.serializer.objectStreamReset', u'100'),
 (u'spark.app.id', u'local-1480405447043'),
 (u'spark.master', u'local[*]'),
 (u'spark.executor.id', u'driver'),
 (u'spark.submit.deployMode', u'client'),
 (u'spark.driver.extraClassPath', u'/usr/share/java/postgresql-jdbc.jar'),
 (u'spark.app.name', u'PySparkShell'),
 (u'hive.metastore.warehouse.dir',
  u'file:/home/gpadmin/spark/spark-warehouse')]

# Query data from the master

In [41]:
from pyspark.sql import DataFrameReader

# Greenplum master
url = 'postgresql://gpdb-mdw:5432/postgres'

properties = {'user': 'gpadmin', 'password': '', 'driver': 'org.postgresql.Driver', 'PGOPTIONS': 'gp_session_role=utility'}
df = DataFrameReader(sqlContext).jdbc(
  url='jdbc:%s' % url, table='sparktest', properties=properties
)

df.show()

+---+
| id|
+---+
| 31|
| 63|
| 95|
|127|
|159|
|191|
|223|
|255|
|260|
|292|
|324|
|356|
|388|
|420|
|452|
|484|
|521|
|553|
|585|
|617|
+---+
only showing top 20 rows



# Define a function to query data in parallel from the segments

In [25]:
from __future__ import generators    # needs to be at the top of your module

recordBatchSize = 100000

def ResultIter(cursor, arraysize):
    'An iterator that uses fetchmany to keep memory usage down'
    while True:
        results = cursor.fetchone() #many(arraysize)
        if not results:
            break
        for result in results:
            yield result

def fetch(host, port, database, query):
    from os import environ
    import psycopg2

    # Allow connections directly to the segments
    environ['PGOPTIONS'] = '-c gp_session_role=utility'

    cnn = psycopg2.connect(host=host, port=port, database=database, user="gpadmin")
    cur = cnn.cursor("serverSideCursor")
    cur.itersize = recordBatchSize
    cur.execute("{0}".format(query))
    
    return cur.fetchall()
    
    #return ResultIter(cur, recordBatchSize)

# Create a list of segment host + port tuples

In [4]:
numNodes = 4
hosts = ["sdw{0}".format(h) for h in xrange(1, numNodes + 1)]

In [5]:
numSegsPerNode = 8
ports = [p for p in xrange(40000, 40000+numSegsPerNode)]

In [6]:
segmentProcesses = [(h, p) for h in hosts for p in ports ]

# Create an RDD containing the hostname, port tuples

In [7]:
rdd = sc.parallelize(segmentProcesses, numNodes*numSegsPerNode)

# Query the segments in parallel

In [33]:
segData = rdd.map(lambda seg: fetch(seg[0], seg[1], "plsprod", "select * from problems_reduced"))

# One list per segment

In [ ]:
%%time

segData.count()

# Merge all segment lists into one

In [32]:
%%time

from itertools import chain

allData = segData.flatMap(lambda xs: chain(*xs))
print allData.count()

879847029
CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 35.2 s


In [ ]:
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils

# Load and parse the data file into an RDD of LabeledPoint.
#allData = MLUtils.loadLibSVMFile(sc, 'data/mllib/sample_libsvm_data.txt')
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = allData.randomSplit([0.7, 0.3])

# Train a RandomForest model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
#  Note: Use larger numTrees in practice.
#  Setting featureSubsetStrategy="auto" lets the algorithm choose.
model = RandomForest.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=3, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=4, maxBins=32)

# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(testData.count())
print('Test Error = ' + str(testErr))
print('Learned classification forest model:')
print(model.toDebugString())